This is the code for analyzing optogenetic expression experiments in Otis, Namboodiri et al. as shown in Figure 5.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import scipy.io as sio
import os
import subprocess
import pickle
from statsmodels.distributions.empirical_distribution import ECDF
import pandas
import statsmodels.formula.api as smf
import statsmodels.api as sm
import statsmodels.stats.multicomp as multi
import statsmodels
import scipy.stats as stats
from sklearn.metrics import roc_auc_score as auROC

In [ ]:
def argsort_forlist(seq):
    #http://stackoverflow.com/questions/3382352/equivalent-of-numpy-argsort-in-basic-python/3382369#3382369
    #by unutbu
    return sorted(range(len(seq)), key=seq.__getitem__)

In [ ]:
# File naming convention is: 'Jim_%s_D%f__0_6000_%s_results.mat' %(Animalname, day of session, 'yymmdd-hhmmss')
# The day of session is coded as a decimal point in the format x.y where x is the total number of days from the start
# and y is the number of days since first day of testing. 

# Each of these matlab files have a whole bunch of internal variables, many of which are only relevant to other
# experiments in the lab. The most important variable if anyone wants access to the raw data is 'eventlog'.
# This contains the raw timestamps and event code (e.g. CS+ is 5 and CS- is 6) for every behavioral event. The
# variables that are relevant to this data are imported into python in the cell below.

In [ ]:
indir = 'full/path/to/directory/where/data/is/stored'

groups = ['NAc', 'PVT']
conditions = ['ChR2','eNpHR3','eYFP']
laserconditions = ['Laser','NoLaser']

nlicksdata_pop = {}

alldata_plus_NAc = pandas.DataFrame(columns=['nlicks','Virus','LaserCondition', 'Animal', 'Run'])
alldata_plus_PVT = pandas.DataFrame(columns=['nlicks','Virus','LaserCondition', 'Animal', 'Run'])
animalids_groups = {}

for group in groups:
    animalids_groups[group] = {}
    for condition in conditions:
        for lasercondition in laserconditions:
            datadir = os.path.join(indir,group+'-'+condition,lasercondition)
            tempmatfiles = os.walk(datadir).next()[2]
            matfiles = [f for f in tempmatfiles if os.path.splitext(f)[1]=='.mat']
            
            initsize = 10000 #Initialization size for number of sessions per condition
            tempdataplus = np.nan*np.ones((initsize,))
            tempdataminus = np.nan*np.ones((initsize,))
            animalid_plusvec = np.array(['NaN']*initsize, dtype='object')
            animalid_minusvec = np.array(['NaN']*initsize, dtype='object')
            runnum_plusvec = np.array(['NaN']*initsize)
            runnum_minusvec = np.array(['NaN']*initsize)
            tempsum = 0
            
            animals = list(set([matfile.split('_')[1] for matfile in matfiles]))
            animalids_groups[group][condition] = animals
            timestamps = {}
            run_nums = {} #Run number for each experiment within an animal. 3 runs per animal in the design
            # This is calculated automatically from the day of session. Sorting the days in ascending order
            # for each condition (Laser or NoLaser) will result in 3 ordered files each in a condition. The
            # files with the same rank are paired and will have the same rank number
            for animal in animals:
                timestamps[animal] = [float(matfile.split('_')[2][1:]) for matfile in matfiles if (animal+'_') in matfile]
                run_nums[animal] = {}
                temp = argsort_forlist(timestamps[animal])
                tempidx = list(temp)
                for i in range(len(temp)):
                    tempidx[temp[i]] = i
                for t, timestamp in enumerate(timestamps[animal]):
                    run_nums[animal][timestamp] = tempidx[t]
                    
            for matfile in matfiles:
                #print group, condition, lasercondition, matfile
                animalid = matfile.split('_')[1]
                
                timeofsession = float(matfile.split('_')[2][1:])
                
                run_num = run_nums[animalid][timeofsession]

                behaviordata = sio.loadmat(os.path.join(datadir, matfile))
                
                # In the MATLAB results files, the 12kHz tone is labeled as CSplus and the 3kHz
                # tone is labeled as CS-. This does not reflect the reward association as that
                # was set by the variables 'csplusprob' and 'csminusprob'. So if 'csplusprob' was
                # 0 and 'csminusprob' was 100, the 12kHz tone was associated with no reward but
                # the 3kHz tone was associated with 100% reward. Thus, please don't confuse the
                # variable names CSplus and CSminus as reflecting the cue-reward contingency.
                try:
                    csplusprob = np.squeeze(behaviordata['csplusprob'])
                    csminusprob = np.squeeze(behaviordata['csminusprob'])    
                except: #If these are not mentioned, they were the default
                    csplusprob = 100
                    csminusprob = 0
                if csplusprob>csminusprob:   # In this case, the reward cue is csplus or 12kHz 
                    nlicksplus = np.squeeze(behaviordata['nlicksplus'])
                    nlicksbaselineplus = np.squeeze(behaviordata['nlicksbaselineplus'])
                    nlicks = np.mean(nlicksplus, axis=1)
                    baseline = nlicksbaselineplus[:,0]
                    #temp = np.around(nlicks-baseline, decimals=3) # All licks
                    temp = np.nanmean(nlicks-baseline) # Mean per session
                    tempdataplus[tempsum:tempsum+temp.size] = temp
                    animalid_plusvec[tempsum:tempsum+temp.size] = np.array([animalid]*temp.size, dtype='object')
                    runnum_plusvec[tempsum:tempsum+temp.size] = np.array([run_num]*temp.size)

                    nlicksminus = np.squeeze(behaviordata['nlicksminus'])
                    nlicksbaselineminus = np.squeeze(behaviordata['nlicksbaselineminus'])
                    nlicks = np.mean(nlicksminus, axis=1)
                    baseline = nlicksbaselineminus[:,-1]
                    #temp = np.around(nlicks-baseline, decimals=3) # All licks
                    temp = np.nanmean(nlicks-baseline) # Mean per session
                    tempdataminus[tempsum:tempsum+temp.size] = temp
                    animalid_minusvec[tempsum:tempsum+temp.size] = np.array([animalid]*temp.size, dtype='object')
                    runnum_minusvec[tempsum:tempsum+temp.size] = np.array([run_num]*temp.size)
                else:   # In this case, the reward cue is csminus or 3kHz 
                    nlicksminus = np.squeeze(behaviordata['nlicksplus'])
                    nlicksbaselineminus = np.squeeze(behaviordata['nlicksbaselineplus'])
                    nlicks = np.mean(nlicksminus, axis=1)
                    baseline = nlicksbaselineminus[:,0]
                    #temp = np.around(nlicks-baseline, decimals=3) # All licks
                    temp = np.nanmean(nlicks-baseline) # Mean per session
                    tempdataminus[tempsum:tempsum+temp.size] = temp
                    animalid_minusvec[tempsum:tempsum+temp.size] = np.array([animalid]*temp.size, dtype='object')
                    runnum_minusvec[tempsum:tempsum+temp.size] = np.array([run_num]*temp.size)

                    nlicksplus = np.squeeze(behaviordata['nlicksminus'])
                    nlicksbaselineplus = np.squeeze(behaviordata['nlicksbaselineminus'])
                    nlicks = np.mean(nlicksplus, axis=1)
                    baseline = nlicksbaselineplus[:,-1]
                    #temp = np.around(nlicks-baseline, decimals=3) # All licks
                    temp = np.nanmean(nlicks-baseline) # Mean per session
                    tempdataplus[tempsum:tempsum+temp.size] = temp
                    animalid_plusvec[tempsum:tempsum+temp.size] = np.array([animalid]*temp.size, dtype='object')
                    runnum_plusvec[tempsum:tempsum+temp.size] = np.array([run_num]*temp.size)                
                
                tempsum += temp.size
            #print '%s-%s: %s Number of sessions = %d'%(group, condition, lasercondition,tempsum)
            animalid_plusvec = animalid_plusvec[np.isfinite(tempdataplus)]
            animalid_minusvec = animalid_minusvec[np.isfinite(tempdataminus)]
            runnum_plusvec = runnum_plusvec[np.isfinite(tempdataplus)]
            runnum_minusvec = runnum_minusvec[np.isfinite(tempdataminus)]
            tempdataplus = tempdataplus[np.isfinite(tempdataplus)]
            tempdataminus = tempdataminus[np.isfinite(tempdataminus)]
            
            nlicksdata_pop[os.path.join(group,condition,lasercondition,'plus')] = tempdataplus
            nlicksdata_pop[os.path.join(group,condition,lasercondition,'minus')] = tempdataminus
            
            if group=='NAc':
                data = np.column_stack([tempdataplus, 
                                        [condition]*tempdataplus.shape[0], 
                                        [lasercondition]*tempdataplus.shape[0],
                                        animalid_plusvec,
                                        runnum_plusvec])
                df = pandas.DataFrame(data=data, columns=['nlicks','Virus','LaserCondition', 'Animal', 'Run'])
                alldata_plus_NAc = alldata_plus_NAc.append(df, ignore_index=True)
            elif group=='PVT':
                data = np.column_stack([tempdataplus, 
                                        [condition]*tempdataplus.shape[0], 
                                        [lasercondition]*tempdataplus.shape[0],
                                        animalid_plusvec,
                                        runnum_plusvec])
                df = pandas.DataFrame(data=data, columns=['nlicks','Virus','LaserCondition', 'Animal', 'Run'])
                alldata_plus_PVT = alldata_plus_PVT.append(df, ignore_index=True)
                
            
with open(os.path.join(indir, 'nlicksdata_population.pickle'), 'wb') as handle:
    pickle.dump(nlicksdata_pop, handle)

In [ ]:
with open(os.path.join(indir, 'nlicksdata_population.pickle'), 'rb') as handle:
    nlicksdata_pop = pickle.load(handle)

Plot the cumulative distribution function of the nlicksplus and nlicksminus variables. This is especially useful if you want to visualize all licks pooled across all animals for each condition. In this case, in the previous cell, uncomment the line saying All licks and comment out the mean licks line. This plot of CDF of all licks is shown in Figure 5d-f and 5j-l

In [ ]:
fig, axs = plt.subplots(2,3, sharex='col', sharey='row',figsize=(10,10))
width = 1
for g, group in enumerate(groups):
    for c, condition in enumerate(conditions):
        for lasercondition in laserconditions:
            nlicksplus = nlicksdata_pop[os.path.join(group,condition,lasercondition,'plus')]
            x=np.argsort(nlicksplus)
            ax = axs[g,c]
            if lasercondition == 'Laser':
                axs[g,c].plot(nlicksplus[x], ECDF(nlicksplus)(nlicksplus)[x],label='Laser+',color='b')
            else:
                axs[g,c].plot(nlicksplus[x], ECDF(nlicksplus)(nlicksplus)[x],label='NoLaser+',color='k')
            nlicksminus = nlicksdata_pop[os.path.join(group,condition,lasercondition,'minus')]
            x=np.argsort(nlicksminus)
            if lasercondition == 'Laser':
                axs[g,c].plot(nlicksminus[x],ECDF(nlicksminus)(nlicksminus)[x],'--', label='Laser-',color='b')
            else:
                axs[g,c].plot(nlicksminus[x],ECDF(nlicksminus)(nlicksminus)[x],'--', label='NoLaser-',color='k')
        axs[g,c].set_title(condition)
        axs[g,c].set_xlim([0, 10])
    pad = 5 # in points
    axs[g,0].annotate(group, xy=(0, 0.5), xytext=(-axs[g,0].yaxis.labelpad - pad, 0),
            xycoords=axs[g,0].yaxis.label, textcoords='offset points',
            size='large', ha='right', va='center')   
fig.tight_layout()
fig.subplots_adjust(left=0.15, top=0.95)

In [ ]:
fig.savefig(os.path.join(indir, 'Expression CDF.pdf'), format='pdf')

Plot the barplot of the nlicksplus and nlicksminus variables. This plot is shown in Figure 5j-l.

In [ ]:
fig, axs = plt.subplots(2,3, sharex='col', sharey='row',figsize=(10,10))
width = 1
for g, group in enumerate(groups):
    for c, condition in enumerate(conditions):
        for lasercondition in laserconditions:
            nlicksplus = nlicksdata_pop[os.path.join(group,condition,lasercondition,'plus')]
            x=np.argsort(nlicksplus)
            ax = axs[g,c]
            if lasercondition == 'Laser':
                ax.bar(4, np.mean(nlicksplus), yerr=stats.sem(nlicksplus),
                       color='b', width=width, ecolor='k')
            else:
                ax.bar(3, np.mean(nlicksplus), yerr=stats.sem(nlicksplus),
                       color='k', width=width, ecolor='k') 
            nlicksminus = nlicksdata_pop[os.path.join(group,condition,lasercondition,'minus')]
            x=np.argsort(nlicksminus)
            if lasercondition == 'Laser':
                ax.bar(1, np.mean(nlicksminus), yerr=stats.sem(nlicksminus),
                       color='b', width=width, ecolor='k') 
            else:
                ax.bar(0, np.mean(nlicksminus), yerr=stats.sem(nlicksminus),
                       color='k', width=width, ecolor='k') 
            ax.text(1,6.5,'CS-')
            ax.text(4,6.5,'CS+')
            ax.set_xticks([a+ width/2.0 for a in [0, 1, 3, 4]])
            ax.set_xticklabels(('NoLaser', 'Laser', 'NoLaser', 'Laser'), rotation='vertical')
            ax.set_ylim([0, 7])
        axs[g,c].set_title(condition)
        #axs[g,c].set_xlim([0, 10])
    axs[g,0].set_ylabel('$\Delta$Licking Rate')
    pad = 5 # in points
    axs[g,0].annotate(group, xy=(0, 0.5), xytext=(-axs[g,0].yaxis.labelpad - pad, 0),
            xycoords=axs[g,0].yaxis.label, textcoords='offset points',
            size='large', ha='right', va='center')   
fig.tight_layout()
fig.subplots_adjust(left=0.15, top=0.95)

In [ ]:
fig.savefig(os.path.join(indir, 'Expression Barplot.pdf'), format='pdf')

Now do all the analyses. In this experiment, it was clear apriori that the comparison of interest was the interaction between laser and virus for the experimental virus groups (ChR2 or eNpHR3) and the control virus group (eYFP). This is done below for each projection population. Of course, we are making multiple comparisons due to there being 2 interactions of interest. So we correct for this with the Benjamini-Hochberg false discovery rate correction.

In [ ]:
def Benjamini_Hochberg_pvalcorrection(vector_of_pvals):
    # This function implements the BH FDR correction
    
    # Parameters:
    # Vector of p values from the different tests
    
    # Returns: Corrected p values.
    
    sortedpvals = np.sort(vector_of_pvals)
    orderofpvals = np.argsort(vector_of_pvals)
    m = sortedpvals[np.isfinite(sortedpvals)].size #Total number of hypotheses
    corrected_sortedpvals = np.nan*np.ones((sortedpvals.size,))
    corrected_sortedpvals[m-1] = sortedpvals[m-1]
    for i in range(m-2, -1, -1):
        corrected_sortedpvals[i] = np.amin([corrected_sortedpvals[i+1], sortedpvals[i]*m/(i+1)])
    correctedpvals = np.nan*np.ones((vector_of_pvals.size,))
    correctedpvals[orderofpvals] = corrected_sortedpvals
    return correctedpvals

In [ ]:
alldata_plus_NAc['nlicks'] = alldata_plus_NAc['nlicks'].astype(float)
group = 'NAc'
lasereffects_pop = {}
for condition in conditions:
    animals = animalids_groups[group][condition]
    temp = []
    for animal in animals:
        runs = list(set(alldata_plus_NAc[(alldata_plus_NAc['Virus']==condition)
                                         & (alldata_plus_NAc['Animal']==animal)]['Run']))
        #print condition, animal, runs
        for run in runs:
            nlicks_laser = np.array(alldata_plus_NAc[(alldata_plus_NAc['Virus']==condition)
                                                     & (alldata_plus_NAc['Animal']==animal)
                                                     & (alldata_plus_NAc['LaserCondition']=='Laser')
                                                     & (alldata_plus_NAc['Run']==run)]['nlicks'])
            nlicks_nolaser = np.array(alldata_plus_NAc[(alldata_plus_NAc['Virus']==condition)
                                                       & (alldata_plus_NAc['Animal']==animal)
                                                       & (alldata_plus_NAc['LaserCondition']=='NoLaser')
                                                       & (alldata_plus_NAc['Run']==run)]['nlicks'])
            temp.append(nlicks_laser-nlicks_nolaser)
    lasereffects_pop[condition] = np.squeeze(np.array(temp))
#print lasereffects_pop

conditions_of_interest = ['eNpHR3', 'ChR2']
results_conditions = np.nan*np.ones((len(conditions_of_interest), 5)) # effect size, auROC, p, n1, n2
for c, condition in enumerate(conditions_of_interest):
    x = lasereffects_pop[condition]
    y = lasereffects_pop['eYFP']
    results_conditions[c,0] = np.median(x)-np.median(y)
    results_conditions[c,1] = auROC(np.concatenate((np.ones(x.size,), np.zeros(y.size,))),
                                    np.concatenate((x, y)))
    results_conditions[c,2] = stats.mannwhitneyu(x, y)[1]
    results_conditions[c,3] = x.size
    results_conditions[c,4] = y.size
    
results_conditions[:,2] = Benjamini_Hochberg_pvalcorrection(results_conditions[:,2])
for c, condition in enumerate(conditions_of_interest):
    print 'The effect of laser in %s when compared to eYFP\n\
    on median lick rate was %f; auROC(%d, %d) = %f, p value=%f ' % (condition,
                                                                results_conditions[c,0],
                                                                results_conditions[c,3],
                                                                results_conditions[c,4],
                                                                results_conditions[c,1],
                                                                results_conditions[c,2])
    print '\n'

In [ ]:
alldata_plus_PVT['nlicks'] = alldata_plus_PVT['nlicks'].astype(float)
group = 'PVT'
lasereffects_pop = {}
for condition in conditions:
    animals = animalids_groups[group][condition]
    temp = []
    for animal in animals:
        runs = list(set(alldata_plus_PVT[(alldata_plus_PVT['Virus']==condition)
                                         & (alldata_plus_PVT['Animal']==animal)]['Run']))
        #print condition, animal, runs
        for run in runs:
            nlicks_laser = np.array(alldata_plus_PVT[(alldata_plus_PVT['Virus']==condition)
                                                     & (alldata_plus_PVT['Animal']==animal)
                                                     & (alldata_plus_PVT['LaserCondition']=='Laser')
                                                     & (alldata_plus_PVT['Run']==run)]['nlicks'])
            nlicks_nolaser = np.array(alldata_plus_PVT[(alldata_plus_PVT['Virus']==condition)
                                                       & (alldata_plus_PVT['Animal']==animal)
                                                       & (alldata_plus_PVT['LaserCondition']=='NoLaser')
                                                       & (alldata_plus_PVT['Run']==run)]['nlicks'])
            temp.append(nlicks_laser-nlicks_nolaser)
    lasereffects_pop[condition] = np.squeeze(np.array(temp))
#print lasereffects_pop

conditions_of_interest = ['eNpHR3', 'ChR2']
results_conditions = np.nan*np.ones((len(conditions_of_interest), 5)) # effect size, U, p, n1, n2
for c, condition in enumerate(conditions_of_interest):
    x = lasereffects_pop[condition]
    y = lasereffects_pop['eYFP']
    results_conditions[c,0] = np.median(x)-np.median(y)
    results_conditions[c,1] = auROC(np.concatenate((np.ones(x.size,), np.zeros(y.size,))),
                                    np.concatenate((x, y)))
    results_conditions[c,2] = stats.mannwhitneyu(x, y)[1]
    results_conditions[c,3] = x.size
    results_conditions[c,4] = y.size
    
results_conditions[:,2] = Benjamini_Hochberg_pvalcorrection(results_conditions[:,2])
for c, condition in enumerate(conditions_of_interest):
    print 'The effect of laser in %s when compared to eYFP\n\
    on median lick rate was %f; auROC(%d, %d) = %f, p value=%f ' % (condition,
                                                                results_conditions[c,0],
                                                                results_conditions[c,3],
                                                                results_conditions[c,4],
                                                                results_conditions[c,1],
                                                                results_conditions[c,2])
    print '\n'